# Introduction - Runnables and Chains (LCEL)

Learn the Langchain Expression Language (LCEL) by composing Runnables into Chains using the | operator. You'll build prompt -> model -> parser flows, stream flows, fan out inputs, run branches in parallel, and compare LCEL to the legacy `SequentialChain`

## Bootstrap

⚓--- Before proceeding futher it is very important you do the following: --- 👾

Select the 🗝 (key) icon in the left pane and include your OpenAI Api key with Name as "OPENAPI_KEY" and value as the key, and grant it notebook access in order to be able to run this notebook.

Run the below two cells in the order they are in, before running further cells. Wait till a number appears in place of '*' or '[ ]'. Below the cell you should see "Ready. LangChain + OpenAI set up."

In [ ]:
!pip install -q langchain langchain-openai langchain-community

In [ ]:
# 1) Imports & env
import time
from google.colab import userdata

key = userdata.get('OPENAI_API_KEY')  # returns None if not granted
if not key:
    raise RuntimeError("Set OPENAI_API_KEY in a .env file next to this notebook.")

# 2) LangChain core
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# 3) OpenAI chat model via LangChain
from langchain_openai import ChatOpenAI

# 4) Streaming helpers
from langchain_core.callbacks import StdOutCallbackHandler, CallbackManager

llm = ChatOpenAI(
    model="gpt-4o-mini",           # pick a small, fast model for learning
    temperature=0.2,
    streaming=True,                # critical for real-time tokens
    # callbacks can be set here or passed per-invoke
    callback_manager=CallbackManager([StdOutCallbackHandler()]),
    api_key=key,
)

print("Ready. LangChain + OpenAI set up.")

## Starting with a Simple LCEL Chain

Here's a simple LCEL chain that utilizes core components from Langchain like PromptTemplates and Output parsers.

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a concise assistant."),
        ("user", "Explain {topic} in 3 bullet points.")
    ]
)

chain = prompt | llm | StrOutputParser()

# Single request (no streaming output capture needed; the callback already prints as tokens)
result = chain.invoke({"topic": "LangChain Expression Language (LCEL)"})
print("\n\nFinal text:\n", result)

### How it works

1. **PromptTemplates**: PromptTemplate enables dynamic insertion of variables into standardized prompt structures. Here, the `ChatPromptTemplate` defines the system/user messages with a placeholder `{topic}`.
2. **Model**: The model is the `llm` we are using, it can be any llm even sourced llms hosted on Ollama. Here in our case, it is ChatGPT 4o mini for which we have a separate library support by Langchain `langchain_openai`. In the configuration of our model in the previous cell, we have set `streaming=true` so tokens appear as they are produced.
3. **OutputParser**: OutputParsers are a way of ensuring the output of the llm is structured in the way you are expecting them, it can vary from unstructured formats like String or structured formats like XML, JSON, CSV, etc. Here we are using `StrOutputParser` which ensures the returned text is a String of text.
4. **LCEL Chain:** LangChain Expression Language (LCEL) is a declarative way of composing chains with different components superceding the traditional `SequentialChain`.

## Streams

Let's say you want to create a chatbot where the output must appear as they are being typed or any other scenario where you want to stream the tokens as they are generated. While we cannot completely stream the tokens as they are generated by the LLMs, we can create an impression like it's being streamed using `.stream`

In [ ]:
# Any LCEL runnable supports .stream() which yields chunks
chunks = chain.stream({"topic": "Runnables vs Chains in LangChain"})
buffer = []
for chunk in chunks:
    # chunk is text here (post StrOutputParser)
    print(chunk, end="", flush=True)
    buffer.append(chunk)
final_text = "".join(buffer)

## Runnable

Before going any further it is imperative you understand the **Runnable** interface that acts as the core interface for Langchain components. It provides a standardized way of interacting with the components.

The Runnable interface defines three main methods for execution:

- `.invoke()`: This method is for synchronous execution. It takes a single input and returns a single output. It's best used when you're running a single chain or need to get a result immediately without parallelism.

- `.batch()`: This method is also for synchronous execution, but it takes a list of inputs and returns a list of outputs. It's highly efficient because it can process all inputs in parallel, which is useful when you have many tasks to complete.

- `.stream()`: This method is for streaming execution. It takes a single input and returns a stream of outputs (an iterator). This is especially useful for large language models (LLMs) and other components that can generate output piece by piece. It allows you to display the results to the user as they are generated, providing a better user experience.

Runnable components can come together to create a chain which is also a Runnable instance. That means, in the code

```python
chain = prompt | llm | StrOutputParser()
```

All the components including the chain itself is a Runnable component.

## Runnable Lambda

RunnableLambda wraps your Python functions as a Runnable.

In [ ]:
def get_time(_):
    return time.strftime("%Y-%m-%d %H:%M:%S")

def normalize_question(x):
    return x["question"].strip().rstrip("?") + "?"

time_r = RunnableLambda(get_time)
norm_q = RunnableLambda(normalize_question)

multi_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful and concise. Current time: {now}"),
    ("user", "{question}")
])

multi_chain = ({"now": time_r, "question": norm_q}) | multi_prompt | llm | StrOutputParser()

result = multi_chain.invoke({"question": " what is a Runnable  "})
print("\n--- Final Output ---")
print(result)

Here, two instances are created:

- `time_r` - wraps `get_time()` to provide current timestamp
- `norm_q` - wraps `normalize_question()` to clean up question formatting

Now, that the Python code has been converted into a Runnable it can be used to compose chains.

### Composing inputs and Dictionaries

If you look the `.invoke` or `.stream` used so far. All of them include a dictionary. This dictionary is passed as the input to the chain upon say `.invoke`.

```python
{"now": time_r, "question": norm_q}
```

In the above case, When you create a dictionary with Runnable values like this, LangChain automatically converts it into a **RunnableParallel**, meaning:
- `time_r` and `norm_q` execute simultaneously (in parallel)
- Both functions receive the same input dictionary: `{"question": " what is a Runnable  "}`

## RunnableParallel to get a Dictionary of results

You can run two (or more) chains in parallel and get a dict of results.

In [ ]:
style_prompt = ChatPromptTemplate.from_messages([
    ("system", "Rewrite the user content in {style} style."),
    ("user", "{content}")
])
style_chain = style_prompt | llm | StrOutputParser()

parallel = RunnableParallel(
    formal=({"style": RunnableLambda(lambda _: "formal"),
             "content": RunnableLambda(lambda x: x["text"])}) | style_chain,
    casual=({"style": RunnableLambda(lambda _: "casual"),
             "content": RunnableLambda(lambda x: x["text"])}) | style_chain,
)

outputs = parallel.invoke({"text": "please explain LCEL briefly"})
print("\n--- FORMAL ---\n", outputs["formal"])
print("\n--- CASUAL ---\n", outputs["casual"])

If you are confused by the RunnableLambda here, It's simply wrapping anonymous functions in Python as Runnable. `lambda` is the keyword to create anonymous functions.

- `lambda _: "formal"` is an anonymous function that ignores any input and presents the output as "formal". _ is the universal symbol in programming of ignoring input parameters. Langchain pipeline always passes data to a Runnable, so a Runnable should be able to accept input parameters even if it is not going to use them.
- `lambda x: x["text"]` is another anonymous function that takes in an input dictionary and returns the value for the key "text" in that dictionary.

Here the chains wrapped around `RunnableParallel` are executed simultaneously and not sequentially. To confirm this yourself you can use `.stream` insteado of invoke here and you will get the chunks parallely and outputting them will show you that "-- FORMAL --" and "-- CASUAL --" appear simultaneously. **Try it as an exercise in the below cell:**
> Hint: The chunks will not be text this time and the output won't be coherent

In [ ]:
stream_outputs = parallel.stream({"text": "please explain LCEL briefly"})

## Your code here

## Map and Batch processing

You can setup a chain to take a list of inputs and return a list of outputs using `.map` at the end of your chain.

In [ ]:
items = []
for t in ["Runnables", "LCEL", "Output Parsers"]:
    items.append({"topic": t}) # Creates a list of dictionaries with key "topic"

mapped = (prompt | llm | StrOutputParser()).map()   # list-in → list-out
map_results = mapped.invoke(items)

print("\n--- MAPPED RESULTS ---")
print("\n---\n".join(map_results))

This can also be done using .batch without .map which also serves a similar purpose (to take a list of inputs and return a list of outputs)

In [ ]:
mapped_batch = prompt | llm | StrOutputParser()
batch_results = mapped_batch.batch(items)
print("\n--- BATCHED RESULTS ---")
print("\n---\n".join(batch_results))

## Legacy SequentialChain vs LCEL

Let's consider the following example:

You have a company idea you need to a company name and a slogan based on the company name and the idea. Let's do this using SequentialChain and LCEL to illustrate the power of LCEL and why declarative way of composing chains is easier.

### SequentialChain

In [ ]:
from langchain.chains import LLMChain, SequentialChain  # legacy-style chains

# Step 1: Company name
prompt_template_name = ChatPromptTemplate.from_messages(
    [
        ("system", "You respond in one word."),
        ("user", "What is a creative and memorable name for a company that sells {product}?")
    ]
)

name_chain = LLMChain(
    llm=llm,
    prompt=prompt_template_name,
    output_key="company_name"
)

# Step 2: Slogan (consumes `company_name`)
prompt_template_slogan = PromptTemplate.from_template(
    template="Write a catchy and short slogan for a company named '{company_name}'.",
)

slogan_chain = LLMChain(
    llm=llm,
    prompt=prompt_template_slogan,
    output_key="slogan"
)

# Step 3: Combine with SequentialChain
sequential_chain = SequentialChain(
    chains=[name_chain, slogan_chain],
    input_variables=["product"],
    output_variables=["company_name", "slogan"],
    verbose=True
)

product_idea = "eco-friendly bamboo toothbrushes"
result = sequential_chain({"product": product_idea})

print("\n--- Final Output ---")
print(f"For a company selling '{product_idea}':")
print(f"Suggested Company Name: {result['company_name'].strip()}")
print(f"Suggested Slogan: {result['slogan'].strip()}")

### LCEL

Make sure you run the previous cell before running the below cell.

In [ ]:
# Step A: a Runnable that produces a company name
company_name = (
    prompt_template_name
    | llm
    | StrOutputParser()
)

# Step B: build a dict state with the tagline
# Then feed it to a second prompt for the slogan
lcel_chain = {
    "company_name": company_name
} | {
    "company_name": RunnableLambda(lambda x: x["company_name"]),
    "slogan": (
        prompt_template_slogan
        | llm
        | StrOutputParser()
    )
}

out = lcel_chain.invoke({"product": "eco-friendly bamboo toothbrushes"})
print("\n--- LCEL OUTPUT ---")
print("Company Name:", out["company_name"].strip())
print("Slogan:", out["slogan"].strip())